In [1]:
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from kerastuner import HyperModel, Hyperband


2.12.1
# GPUs Available:  1


In [2]:
# Experiment 5
# changes from experiment 2 (Attempt at MTL again in light that x1 x2 swap does improves learning)
# there is only 1 leakage ness factor - The assumption is that there is always atleast one leakage when the model is being used.
# Leakageness factor confirm if there exist 1 additional leakage as well.
# 1. interchanged x1 and x2 based on their Value
# removed column leak_1
# create a model 5 outputs
# use the concept of masking

In [3]:
model_path = 'saved_model/Multi_leak/2_loss/Mid_structure_anchor/5 Output_to_transfer_withswap_mask/'
project_name='multi_task_tuning_5coords_withswap_mask'

In [4]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
# two_leakage["leak_1"] = 1
# two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
# single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data['x2'] = data['x2'].replace(np.nan, 0)
data['y2'] = data['y2'].replace(np.nan, 0)

In [5]:
data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'tot_residual_flow', 
       'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2']]
x = data.drop(['x1', 'y1', 'x2', 'y2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 


# sort x1, y1 and x2, y2. coordinates with lowest x will take the position of x1
def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1


y1_data = [y_train, y_val, y_test]
y1_data_types = ['y_train', 'y_val', 'y_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

y1_train = y_train[['x1', 'y1']]
y1_test = y_test[['x1', 'y1']]
y1_val = y_val[['x1', 'y1']]

y2_train = y_train[['x2', 'y2']]
y2_test = y_test[['x2', 'y2']]
y2_val = y_val[['x2', 'y2']]

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

scaler_coords2 = StandardScaler()
y2_train = scaler_coords2.fit_transform(y2_train)
y2_test = scaler_coords2.transform(y2_test)
y2_val = scaler_coords2.transform(y2_val)

y2_train = pd.DataFrame(y2_train, columns=y2_columns)
y2_test = pd.DataFrame(y2_test, columns=y2_columns)
y2_val = pd.DataFrame(y2_val, columns=y2_columns)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

X_train = pd.DataFrame(X_train, columns=X_columns)
X_test = pd.DataFrame(X_test, columns=X_columns)
X_val = pd.DataFrame(X_val, columns=X_columns)

y_train = [y1_train, y2_train]
y_val = [y1_val, y2_val]
y_test = [y1_test, y2_test]

In [6]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
EPOCHS = 1000

# Define custom loss function
def masked_mse(y_true, y_pred):
    mask = keras.backend.cast(keras.backend.not_equal(y_true, 0), keras.backend.floatx())
    mse = keras.backend.mean(keras.backend.square(y_true - y_pred) * mask)
    return mse

# Define model builder function for Keras Tuner
def model_builder(hp):
    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)

    # Define model
    inputs = Input(shape=(10,))
    shared_layer = inputs
    for i in range(hp.Int('num_layers', 1, 15)):
        shared_layer = layers.Dense(
            units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", ["relu", 'elu']),
            # add elu
            kernel_initializer='he_uniform'
        )(shared_layer)

    # Output 1
    out1 = Dense(32, activation='relu')(shared_layer)
    out1 = Dense(2)(out1)

    # Output 2
    out2 = Dense(32, activation='relu')(shared_layer)
    out2 = Dense(2)(out2)

    model = Model(inputs=inputs, outputs=[out1, out2])

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

    # model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss=['mse', masked_mse])
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=hp_learning_rate),
                    loss=['mse', masked_mse], metrics=['mae'])
    
    return model

tuner = Hyperband(
    model_builder,
    objective =  kt.Objective("val_loss", direction="min"),
    max_epochs=EPOCHS,
    factor=2,
    directory="../../tensorflow_log_files/studienarbeit/",
    project_name=project_name
)

# Search for the best hyperparameters
tuner.search(X_train, y_train, validation_data=(X_val, y_val), 
             verbose = 1, epochs=EPOCHS, shuffle = True)


best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hps)

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)


# Train the best model on the full dataset
best_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
             verbose = 1, epochs=EPOCHS, shuffle = True, batch = 32)

INFO:tensorflow:Reloading Tuner from ../../tensorflow_log_files/studienarbeit/multi_task_tuning_5coords_withswap_mask/tuner0.json

Search: Running Trial #2592

Value             |Best Value So Far |Hyperparameter
384               |512               |units
11                |4                 |num_layers
448               |288               |units_0
elu               |elu               |activation
0.00069671        |0.0018322         |lr
352               |128               |units_1
128               |416               |units_2
32                |256               |units_3
352               |384               |units_4
256               |448               |units_5
384               |288               |units_6
96                |160               |units_7
288               |64                |units_8
480               |480               |units_9
32                |320               |units_10
160               |128               |units_11
96                |128               |units_12
64 

In [ ]:
best_model.summary()
best_model.save(model_path)

In [ ]:
# experiment 6 - use this same idea and increase the output by 1 to include the label of additional leakage